In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#Data Loading & Preprocessing

transform = transforms.Compose(
    [transforms.Resize(224, interpolation=2),#Resize CIFAR 10 Data to appropriate size (for pretrianed nn)
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size_ = 16

#Test 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_, shuffle=True, num_workers=2)

#Val
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataloaders = {'train' : trainloader, 'val' : testloader }
dataset_sizes = {'train': len(trainset), 'val': len(testset)}



In [ ]:
#Try viewing some images

def imshow(img):
    img = img/2 + 0.5 #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

dataiter = iter(trainloader)    #brings batch_size_ many images at one time
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
plt.show()

In [ ]:
#Models & Criterion 

'''
#AlexNet
alexnet = models.alexnet(pretrained=True)
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6]=nn.Linear(num_ftrs, len(classes))
alexnet = alexnet.to(device)

'''
#VGG16
vgg16 = models.vgg16(pretrained=True)
num_ftrs = vgg16.classifier[6].in_features
vgg16.classifier[6]=nn.Linear(num_ftrs, len(classes))
vgg16 = vgg16.to(device)

'''
#Resnet18
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features #number of output nodes
resnet18.fc = nn.Linear(num_ftrs, len(classes))
resnet18 = resnet18.to(device)

'''
#Criterion
criterion = nn.CrossEntropyLoss()

#Optimizer & LR_Scheculer
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
from torch.utils.tensorboard import SummaryWriter

#Train Model
writer = SummaryWriter()
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time() #State starting time
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            #Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad() #zero the parameter gradients
                
                #forward, track history iff in train phase
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs).cuda()
                    _, preds = torch.max(outputs, 1) # 1 class that has maximum score
                    loss = criterion(outputs, labels)
                    
                    #backward + optimize only iff in train phase
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                    else:
                        preds=preds.cpu()
                        
                #statistics 
                running_loss += loss.item() * inputs.size(0)
                preds = preds.cuda()
                running_corrects += torch.sum(preds==labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                scheduler.step()
                writer.add_scalar("Loss/train", epoch_loss, epoch)
                writer.add_scalar("Accuracy/train", epoch_acc, epoch)
            
            if phase == 'val':
                writer.add_scalar("Loss/val", epoch_loss, epoch)
                writer.add_scalar("Accuracy/val", epoch_acc, epoch)
                
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print('Epoch {} finished in {}s' .format(epoch, time.time()-epoch_start_time))
        print()

    time_elapsed = time.time() - since
    print('Total Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

print('========= FINISHED MODELING =========')
writer.flush()
writer.close()

In [ ]:
#vgg16 with Adam-0.001
vgg16 = train_model(vgg16, criterion, optimizer, exp_lr_scheduler,num_epochs=30)